# Practica Spark Streaming + Kafka

### Pasos previos

En primer lugar arrancamos Zookeeper

    bin/zookeeper-server-start.sh config/zookeeper.properties
y Kafka

    bin/kafka-server-start.sh config/server.properties
Creamos el topic *QATAR_GP_2014* con la siguiente orden:

    bin/kafka-topics.sh --create --zookeeper localhost:2181 --replication-factor 1 --partitions 1 --topic QATAR_GP_2014
Ponemos en marcha el productor de Kafka

    python kafka_producer.py 0.1 0.3 QATAR_GP_2014 ./data/DATASETMotoGP-Qatar.csv


In [1]:
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

In [17]:
sc = spark.sparkContext
ssc = StreamingContext(sc, 5)

In [18]:
kafkaParams = {"metadata.broker.list": "localhost:9092"}
directKafkaStream = KafkaUtils.createDirectStream(ssc, ["QATAR_GP_2014"], kafkaParams)

### A) Calcular el número total de menciones recibidas por cada cuenta de usuario durante el intervalo de 5 segundos.

In [19]:
lines = directKafkaStream.map(lambda x: x[1])

counts = lines.map(lambda line: line.split(",")) \
              .map(lambda line: line[3]) \
              .flatMap(lambda line: line.split(",")) \
              .filter(lambda line: line != '') \
              .map(lambda line: (line,1)) \
              .reduceByKey(lambda a, b: a+b)

counts.pprint()

### B) Calcular la frecuencia total acumulada de apariciones de cada hashtag en el campo body, actualizando un ranking con los 5 hashtags con mayor frecuencia de aparición.

In [112]:
def update_func(new_val, last_sum):
    return sum(new_val) + (last_sum or 0)
ssc.checkpoint("checkpoint")  # checkpoint directory, failure recovery

lines = directKafkaStream.map(lambda x: x[1])
counts = lines.map(lambda line: line.split(",")) \
              .map(lambda line: line[6]) \
              .flatMap(lambda line: line.split(" ")) \
              .filter(lambda x: x.startswith('#')) \
              .map(lambda line: (line, 1)) \
              .reduceByKey(lambda a, b: a+b) \
              .updateStateByKey(update_func) \
              .transform(lambda rdd: rdd.sortBy(lambda x: -x[1]))
                
counts.pprint(5)

### C) Calcular en una ventana temporal 20 segundos con offset de 10 segundos la frecuencia de aparición de cada uno de los 3 posibles tipos de tweets (TW-RT-MT).

In [14]:
ssc.checkpoint("checkpoint")  # checkpoint directory, failure recovery

lines = directKafkaStream.map(lambda x: x[1])
counts = lines.map(lambda line: line.split(",")) \
              .filter(lambda line: line[9] == "MT" or line[9] =="RT" or line[9] =="TW") \
              .map(lambda line: (line[9], 1)) \
              .reduceByKeyAndWindow(lambda x, y: x+y, lambda x, y: x-y, 20, 10)

counts.pprint()

### Start/stop Spark Streaming context

In [20]:
ssc.start()

-------------------------------------------
Time: 2017-06-14 13:39:30
-------------------------------------------

-------------------------------------------
Time: 2017-06-14 13:39:35
-------------------------------------------



In [16]:
ssc.stop(False)